In [43]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style= "darkgrid", color_codes = True)
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input
from keras.regularizers import l2
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.preprocessing.image import  load_img
import warnings
import glob
import cv2
import pandas as pd
from sklearn.metrics import classification_report
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import pickle

In [22]:
# Define image size and other parameters
img_width, img_height = 256, 256
batchsize = 32
epochs = 4
num_of_class = 2

validation = keras.utils.image_dataset_from_directory(
    directory='Resources/val',
    labels='inferred',
    label_mode='categorical',
    batch_size=batchsize,
    # color_mode='grayscale',
    image_size=(img_width, img_height),
    seed=123)

Found 16 files belonging to 2 classes.


In [23]:
test = keras. utils.image_dataset_from_directory(
    directory='Resources/test',
    labels='inferred',
    label_mode='categorical',
    batch_size=batchsize,
    # color_mode='grayscale',
    image_size=(img_width, img_height),
    seed=123)

Found 624 files belonging to 2 classes.


In [24]:
train = keras. utils.image_dataset_from_directory(
    directory='Resources/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=batchsize,
    # color_mode='grayscale',
    image_size=(img_width, img_height),
    seed=123)

Found 5216 files belonging to 2 classes.


In [25]:
print(train.class_names)
print(validation.class_names)
print(test.class_names)

['NORMAL', 'PNEUMONIA']
['NORMAL', 'PNEUMONIA']
['NORMAL', 'PNEUMONIA']


In [26]:
x_train = []
y_train = []

for feature, label in train:
    x_train.append(feature.numpy())
    y_train.append(label.numpy())

x_train = np.concatenate(x_train, axis=0)
y_train = np.concatenate(y_train, axis=0)

y_train_df=pd.DataFrame(y_train, columns=['Normal', 'Pneumonia'])
y_train_np=y_train_df['Pneumonia'].to_numpy()

In [27]:
x_train_sample=x_train[0:100]
y_train_sample=y_train_np[0:100]

In [28]:
x_val = []
y_val = []
    
for feature, label in validation:
    x_val.append(feature.numpy())
    y_val.append(label.numpy())

x_val = np.concatenate(x_val, axis=0)

y_val = np.concatenate(y_val, axis=0)

y_val_df=pd.DataFrame(y_val, columns=['Normal', 'Pneumonia'])
y_val_np=y_val_df['Pneumonia'].to_numpy()

In [ ]:
class_names = validation.class_names
print("Class names:", class_names)
for images, labels in validation.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)
    print("Labels (one-hot):", labels.numpy())
# Step 5: Display some images and their one-hot encoded labels
plt.figure(figsize=(10, 10))
for images, labels in validation.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Label: {labels[i].numpy()}")
        plt.axis("off")
plt.show()

In [29]:
x_test = []
y_test = []

for feature, label in test:
    x_test.append(feature.numpy())
    y_test.append(label.numpy())

x_test = np.concatenate(x_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

y_test_df=pd.DataFrame(y_test, columns=['Normal', 'Pneumonia'])
y_test_np=y_test_df['Pneumonia'].to_numpy()

In [30]:
# check the shapes of 'x_train' and 'y_train':
print("Shape of 'x_train':", x_train.shape)
print("Shape of 'y_train_np':", y_train_np.shape)
print("Shape of 'x_val':", x_val.shape)
print("Shape of 'y_val_np':", y_val_np.shape)
print("Shape of 'x_test':", x_test.shape)
print("Shape of 'y_test_np':", y_test_np.shape)

Shape of 'x_train': (5216, 256, 256, 3)
Shape of 'y_train_np': (5216,)
Shape of 'x_val': (16, 256, 256, 3)
Shape of 'y_val_np': (16,)
Shape of 'x_test': (624, 256, 256, 3)
Shape of 'y_test_np': (624,)


In [31]:
x_train=x_train/255
x_val=x_val/255
x_test=x_test/255

In [ ]:
#!pip install h5py

In [ ]:
import h5py

In [ ]:
# from tensorflow.keras.utils import to_categorical
#  Assuming y_train contains labels 0 and 1
y_train_sample = to_categorical(y_train_sample, num_classes=2)
# y_train = to_categorical(y_train, num_classes=2)
# y_train = to_categorical(y_train, num_classes=2)

In [39]:
# We use transfer learning with VGG16 as the base model
def CNN_Model():
    base_model = VGG16(weights='imagenet', include_top = False, input_shape=(img_width, img_height, 3))
    # Freeze the base model
    for layer in base_model.layers:
        layer.trainable = False
    for i in range(3):
        base_model.layers[-2-i].trainable = True
    CNN = Sequential()
    CNN.add(Input(shape=(img_width, img_height,3)))
    CNN.add(base_model)
    CNN.add(Flatten())
    CNN.add(Dropout(0.3))
    CNN.add(Dense(128, activation='relu', kernel_regularizer=l2(0.05)))
    CNN.add(Dropout(0.2))
    CNN.add(Dense(1, activation='sigmoid'))
    return CNN
# Training The CNN
model = CNN_Model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
# Visualized Layers of CNN
# keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 8, 8, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │       4,194,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,909,249 (72.13 MB)

 Trainable params: 11,273,985 (43.01 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [40]:
hist = model.fit(x_train_sample, y_train_sample, epochs= 6, validation_data= (x_val, y_val_np))
print(hist.history)
best_score = max(hist.history['val_accuracy'])
print(f"Best Validation score is: {best_score}")

Epoch 1/6
4/4 ━━━━━━━━━━━━━━━━━━━━ 59s 13s/step - accuracy: 0.5993 - loss: 19.6396 - val_accuracy: 0.5000 - val_loss: 9.3407
Epoch 2/6
4/4 ━━━━━━━━━━━━━━━━━━━━ 46s 11s/step - accuracy: 0.7717 - loss: 19.3341 - val_accuracy: 0.5000 - val_loss: 6.2465
Epoch 3/6
4/4 ━━━━━━━━━━━━━━━━━━━━ 43s 10s/step - accuracy: 0.6010 - loss: 7.0300 - val_accuracy: 0.5000 - val_loss: 5.7613
Epoch 4/6
4/4 ━━━━━━━━━━━━━━━━━━━━ 43s 10s/step - accuracy: 0.8323 - loss: 5.7576 - val_accuracy: 0.5000 - val_loss: 5.7694
Epoch 5/6
4/4 ━━━━━━━━━━━━━━━━━━━━ 41s 10s/step - accuracy: 0.7879 - loss: 5.6709 - val_accuracy: 0.5000 - val_loss: 5.6940
Epoch 6/6
4/4 ━━━━━━━━━━━━━━━━━━━━ 45s 11s/step - accuracy: 0.8303 - loss: 5.7485 - val_accuracy: 0.5000 - val_loss: 5.4319
{'accuracy': [0.6700000166893005, 0.7599999904632568, 0.6299999952316284, 0.8100000023841858, 0.7900000214576721, 0.8100000023841858], 'loss': [18.086713790893555, 15.8515043258667, 6.7726850509643555, 5.7569684982299805, 5.6362810134887695, 5.7720789909

In [41]:
model.evaluate(x_test, y_test_np)

20/20 ━━━━━━━━━━━━━━━━━━━━ 210s 11s/step - accuracy: 0.6409 - loss: 5.4210


[5.422791004180908, 0.625]

In [42]:
x_test_predict=model.predict(x_test)
print(x_test_predict.shape)
pred_final=np.argmax(x_test_predict,axis=1)
print(pred_final.shape)
print(pred_final)

20/20 ━━━━━━━━━━━━━━━━━━━━ 220s 11s/step
(624, 1)
(624,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [44]:
print(classification_report(y_test_np, pred_final))

              precision    recall  f1-score   support

         0.0       0.38      1.00      0.55       234
         1.0       0.00      0.00      0.00       390

    accuracy                           0.38       624
   macro avg       0.19      0.50      0.27       624
weighted avg       0.14      0.38      0.20       624



C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pred_final